# Comparison with popular libraries

In [1]:
import sys
import subprocess
from antidote import __version__, is_compiled
print(f"""
== Python ==
{sys.version}

== Antidote =
{__version__} {'(cython)' if is_compiled() else ''}
""")


== Python ==
3.9.1 (default, Dec  7 2020, 22:33:43) 
[GCC 9.3.0]

== Antidote =
0.12.1 (cython)



In [2]:
cat /proc/cpuinfo | grep 'model name' | head -n 1

model name	: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz


## Injection Benchmarks

Testing how fast @inject is for several libraries.

In [3]:
import inspect
import re

timings = dict()

def bench(library, injections):
    assert len(injections) == 5
    print(library)
    results = []
    for f in injections:
        # friendly formatting of the function
        args = str(inspect.signature(f)).replace('typing.', '')
        args = re.sub(r'__main__(?:.*?)\.(\w+(?:,\s|\)|\s=))',  r'\1', args)
        args = re.sub(r'<(.*?)\.(\w+) object at \w+>',  r'<\1 object ...>', args)
        args = re.sub(r', (s\d?: )', r',\n' + (1 + len(f.__name__)) * ' ' + r'\1', args)
        print(f"\n\n{f.__name__}{args}\n")
        # timeit
        res = %timeit -o f()
        results.append(res)
    timings[library] = results
    
class ExternalSingleton:
    pass

class ExternalStatus:
    pass

In [4]:
from typing import Annotated
from antidote import Service, inject, factory, Provide, From, __version__, is_compiled
assert is_compiled()
title = f"Antidote: {__version__} (compiled)"

class AntidoteSingleton(Service):
    pass

class AntidoteStatus(Service):
    __antidote__ = Service.Conf(singleton=False)

@factory
def singleton_factory() -> ExternalSingleton:
    return ExternalSingleton()

@factory(singleton=False)
def status_factory() -> ExternalStatus:
    return ExternalStatus()

@inject
def antidote_f1(s: Provide[AntidoteSingleton]):
    return s

@inject
def antidote_f2(s: Annotated[ExternalSingleton, From(singleton_factory)]):
    return s

@inject
def antidote_f3(s: Provide[AntidoteStatus]):
    return s

@inject
def antidote_f4(s: Annotated[ExternalStatus, From(status_factory)]):
    return s

@inject
def antidote_f5(s1: Provide[AntidoteSingleton], 
                s2: Annotated[ExternalSingleton, From(singleton_factory)], 
                s3: Provide[AntidoteStatus],
                s4: Annotated[ExternalStatus, From(status_factory)]):
    return s1, s2, s3, s4
    

bench(f"Antidote: {__version__} (compiled)", [antidote_f1, antidote_f2, antidote_f3, antidote_f4, antidote_f5])

Antidote: 0.12.1 (compiled)


antidote_f1(s: Annotated[AntidoteSingleton, <antidote.core.annotations object ...>])

198 ns ± 4.86 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


antidote_f2(s: Annotated[ExternalSingleton, From(source=<antidote._factory object ...>)])

187 ns ± 3.2 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


antidote_f3(s: Annotated[AntidoteStatus, <antidote.core.annotations object ...>])

335 ns ± 3.28 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


antidote_f4(s: Annotated[ExternalStatus, From(source=<antidote._factory object ...>)])

384 ns ± 12.9 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


antidote_f5(s1: Annotated[AntidoteSingleton, <antidote.core.annotations object ...>],
            s2: Annotated[ExternalSingleton, From(source=<antidote._factory object ...>)],
            s3: Annotated[AntidoteStatus, <antidote.core.annotations object ...>],
            s4: Annotated[ExternalStatus, From(source=

In [5]:
def build_python_inject_bench():
    import inject
    title = f"Python Inject: {inject.__version__}"

    class Singleton:
        pass
    
    class Status:
        pass
    
    def singleton_factory():
        return ExternalSingleton()
    
    def status_factory():
        return ExternalStatus()

    def config(binder):
        binder.bind(Singleton, Singleton)
        binder.bind(ExternalSingleton, singleton_factory)
        binder.bind_to_provider(ExternalStatus, status_factory)
        
    inject.configure(config)

    @inject.autoparams()
    def f1(s: Singleton):
        return s

    @inject.autoparams()
    def f2(s: ExternalSingleton):
        return s

    @inject.autoparams()
    def f3(s: Status):
        return s

    @inject.autoparams()
    def f4(s: ExternalStatus):
        return s

    @inject.autoparams()
    def f5(s1: Singleton, 
           s2: ExternalSingleton, 
           s3: Status,
           s4: ExternalStatus):
        return s1, s2, s3, s4
    
    return title, [f1, f2, f3, f4, f5]

bench(*build_python_inject_bench())

Python Inject: 4.3.1


f1(s: Singleton)

1.87 µs ± 42.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


f2(s: ExternalSingleton)

1.98 µs ± 42.8 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


f3(s: Status)

1.97 µs ± 24.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


f4(s: ExternalStatus)

2.07 µs ± 47.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


f5(s1: Singleton,
   s2: ExternalSingleton,
   s3: Status,
   s4: ExternalStatus)

3.54 µs ± 26.3 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [6]:
def build_injector_bench():
    from injector import Injector, inject, singleton, Module, provider, __version__
    from functools import partial, wraps

    title = f"Injector:  {__version__}"
    
    @singleton
    class Singleton:
        pass
    
    class Status:
        pass
    
    @singleton
    @provider
    def singleton_factory() -> ExternalSingleton:
        return ExternalSingleton()
    
    @provider
    def status_factory() -> ExternalStatus:
        return ExternalStatus()

    @inject
    def f1(s: Singleton):
        return s

    @inject
    def f2(s: ExternalSingleton):
        return s

    @inject
    def f3(s: Status):
        return s

    @inject
    def f4(s: ExternalStatus):
        return s

    @inject
    def f5(s1: Singleton, 
           s2: ExternalSingleton, 
           s3: Status,
           s4: ExternalStatus):
        return s1, s2, s3, s4
    
    injector = Injector()
    return title, [
        wraps(f)(partial(injector.call_with_injection, f))
        for f in [f1, f2, f3, f4, f5]
    ]

bench(*build_injector_bench())

Injector:  0.18.4


f1(s: Singleton)

30.3 µs ± 546 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


f2(s: ExternalSingleton)

31.7 µs ± 1.05 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


f3(s: Status)

31.9 µs ± 381 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


f4(s: ExternalStatus)

31.3 µs ± 413 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


f5(s1: Singleton,
   s2: ExternalSingleton,
   s3: Status,
   s4: ExternalStatus)

50.5 µs ± 853 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [7]:
def build_lagom_bench():
    from lagom import Container, Singleton, dependency_definition, magic_bind_to_container, __version__
    from functools import partial
    
    title = f"Lagom: {__version__}"

    class BenchSingleton:
        pass
    
    class Status:
        pass
    
    def singleton_factory() -> ExternalSingleton:
        return ExternalSingleton()
    
    container = Container()
    container[BenchSingleton] = BenchSingleton()
    container[Status] = Status
    container[ExternalSingleton] = Singleton(singleton_factory)
    
    @dependency_definition(container)
    def status_factory() -> ExternalStatus:
        return ExternalStatus()

    @magic_bind_to_container(container)
    def f1(s: BenchSingleton):
        return s

    @magic_bind_to_container(container)
    def f2(s: ExternalSingleton):
        return s

    @magic_bind_to_container(container)
    def f3(s: Status):
        return s

    @magic_bind_to_container(container)
    def f4(s: ExternalStatus):
        return s

    @magic_bind_to_container(container)
    def f5(s1: BenchSingleton, 
           s2: ExternalSingleton, 
           s3: Status,
           s4: ExternalStatus):
        return s1, s2, s3, s4
    
    return title, [f1, f2, f3, f4, f5]

bench(*build_lagom_bench())

Lagom: 1.0.0


f1(s: BenchSingleton)

5.45 µs ± 154 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


f2(s: ExternalSingleton)

5.63 µs ± 246 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


f3(s: Status)

9.83 µs ± 156 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


f4(s: ExternalStatus)

5.62 µs ± 176 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


f5(s1: BenchSingleton,
   s2: ExternalSingleton,
   s3: Status,
   s4: ExternalStatus)

13.6 µs ± 295 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [8]:
import sys
from dependency_injector import containers, providers, __version__
from dependency_injector.wiring import inject, Provide

class DiSingleton:
    pass

class DiStatus:
    pass
    
def di_singleton_factory():
    return ExternalSingleton()

def di_status_factory():
    return ExternalStatus()


class DiContainer(containers.DeclarativeContainer):
    status = providers.Factory(DiStatus)
    singleton = providers.Singleton(DiSingleton)
    external_status = providers.Factory(di_status_factory)
    external_singleton = providers.Singleton(di_singleton_factory)

@inject
def di_f1(s: DiSingleton = Provide[DiContainer.singleton]):
    return s

@inject
def di_f2(s: ExternalSingleton = Provide[DiContainer.external_singleton]):
    return s

@inject
def di_f3(s: DiStatus = Provide[DiContainer.status]):
    return s

@inject
def di_f4(s: ExternalStatus = Provide[DiContainer.external_status]):
    return s

@inject
def di_f5(s1: DiSingleton = Provide[DiContainer.singleton], 
          s2: ExternalSingleton = Provide[DiContainer.external_singleton], 
          s3: DiStatus = Provide[DiContainer.status],
          s4: ExternalStatus = Provide[DiContainer.external_status]):
    return s1, s2, s3, s4


di_container = DiContainer()
di_container.wire(modules=[sys.modules[__name__]])

bench(f"Dependency Injector: {__version__}", [di_f1, di_f2, di_f3, di_f4, di_f5])

Dependency Injector: 4.14.0


di_f1(s: DiSingleton = <dependency_injector.wiring object ...>)

741 ns ± 5.26 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


di_f2(s: ExternalSingleton = <dependency_injector.wiring object ...>)

782 ns ± 20 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


di_f3(s: DiStatus = <dependency_injector.wiring object ...>)

2.3 µs ± 21.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


di_f4(s: ExternalStatus = <dependency_injector.wiring object ...>)

2.34 µs ± 25.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


di_f5(s1: DiSingleton = <dependency_injector.wiring object ...>,
      s2: ExternalSingleton = <dependency_injector.wiring object ...>,
      s3: DiStatus = <dependency_injector.wiring object ...>,
      s4: ExternalStatus = <dependency_injector.wiring object ...>)

4.46 µs ± 13.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [ ]:
# Not keeping image in Git.
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd

f,(top_ax,bot_ax) = plt.subplots(2,1, sharex=True, figsize=(8,5), dpi=110)

columns = [name 
           for name, _ 
           in sorted(((name, ts[-1].average) 
                       for name, ts 
                       in timings.items()), 
                     key=lambda x: x[1])]

df = pd.DataFrame(
    data = {
        library: [t.average * 1e6 for t in ts]
        for library, ts in timings.items()
    },
    index = ['singleton service', 'singleton factory', 'service', 'factory', 'all']
)[columns]

df.plot.bar(ylim=[7, 70], ax=top_ax)
df.plot.bar(ylim=[0, 7], ax=bot_ax, legend=False, ylabel='Time (µs)', xlabel='Injection case')

top_ax.spines['bottom'].set_visible(False)
bot_ax.spines['top'].set_visible(False)

top_ax.set_title("Comparison benchmark")
plt.tight_layout()


## Isolated container Benchmark

Here we test how costly it is to create an isolated container with existing dependencies. We see that dependency_injector is two orders of magnitude slower than Antidote.

In [10]:
def isolated_di_container():
    container = DiContainer()
    container.wire(modules=[sys.modules[__name__]])
    # yield and execute test code
    container.unwire()

%timeit isolated_di_container()

1.18 ms ± 28.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [11]:
from antidote import world

def isolated_antidote_container():
    with world.test.clone():
        pass  # yield and execute test code

%timeit isolated_antidote_container()

10 µs ± 91.9 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
